In [2]:
# gmailer init


In [83]:
import datetime as dt

import yaml

import requests
import smtplib
import ssl
import mimetypes
from email.message import EmailMessage

import pandas as pd



## Load Configurations

In [92]:
## password and config
config = "../config.yaml"
config = yaml.safe_load(open(config))

def parse_config(config: dict):
    """
    Parse the configuration yaml.

    Parameters
    -------
    config (dict{any}): Dictionary of congiruations.

    Returns
    -------
    credentials (dict(any)): Dictionary of credentials.
    """
    credentials = config["credentials"]
    ...
    return credentials

credentials = parse_config(config)

# Methods

In [93]:
def get_csv_as_list(filepath: str) -> list:
    """
    Get list using a path to a csv.

    Parameters
    -------
    filepath (str): filepath. 

    Returns
    -------
    ([str]): List of values.
    
    """
    import pandas as pd
    recipients = pd.read_csv(recipients_csv_path, header=None)
    return recipients[0].values.tolist()


#### Potential Class 0
class Clearbit():
    """A class for working with Clearbit API."""
    def get_name(response: requests.models.Response) -> (str, str):
        """
        Parse response for HTTP request.

        Parameters
        -------
        response (requests.models.Response): Http response with person data.

        Returns
        -------
        name (str, str): Tuple of first and last name. 
        """
        if not isinstance(y, requests.models.Response):
            print("Not a request.")
            return 
        name = (None, None)
        data = response.json()
        person = data["person"]
        first_name = person["name"]["givenName"]
        last_name = person["name"]["familyName"]
        name = (first_name, last_name)
        return name


    def get_names_from_email_list(recipients_list:[str], username=None, password=None, api_key=None):
        """
        Given a list of recipients, use Clearbit to retreive their names. Other data may be retreieved but at a later stage. 
        The username is the api_key from Clearbit. Read their docs for more info.

        Parameters
        -------
        username (str): Optional. Username.
        password (str): Optional. Password.
        api_key (str): API key.

        Returns
        -------
        names {}: Dict of emails to names.
        """
        names = {}
        # NOTE: may want to batch this in the future or too many requests will be attempted too quickly.
        for email in recipients_list:
            url = f"https://person.clearbit.com/v2/combined/find?email=:{email}"
            clearbit_response = get_response(url, username=clearbit_api_key, password=None, api_key=None)
            names[email] = get_name(clearbit_response)
        return names
#####


def get_response(url: str, username=None, password=None, api_key=None):
    """
    Get a response from API using HTTP.

    Parameters
    -------
    url (str): Url for API request.
    username (str): Optional. Username.
    password (str): Optional. Password.
    api_key (str): API key.

    Returns
    -------
    api_response (requests.models.Response): Response.
    """
    api_response = None
    api_response = requests.get(url, auth=(username, password))
    return api_response





# Email class

In [91]:
sender = "jaime.meriz13@gmail.com"

# recipient path should be: list of recipients. Single recipient.
# recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients.csv"
recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients_2.csv"
recipients = get_csv_as_list(recipients_csv_path)

subject = "DEV" + "-" "Jmail" + "-" + str(dt.date.today())
body = "body\nyours\nDr. Lopez"
# password = "password"

In [131]:
def send_mail(sender: str, recipients: list, subject:str, body: str, password: str, attachment_path=None):
    """
    Send an email via SMTP.

    Parameters
    -------
    sender (str): Sender as a string.
    recipients ([str]): List of recipients. 
    subject (str): Email subject line.
    body (str): Email body as string.
    password (str): Gmail password as string.
    attachment_path (str): Optional. Filepath to attachment. 
    
    Returns
    -------
    None
    """
    def add_attachment(email: EmailMessage, filepath: str) -> EmailMessage:
        """
        Given original email message. May or may not include an attachment already.

        Parameters:
        -------
        email (EmailMessage): EmailMessage.
        filepath (str): Filepath to attachment.

        Returns
        -------
        email (EmailMessage): Mail message with new attachment.
        """
        # Attachments
        with open(filepath, "rb") as fp:
            data = fp.read()

        # guess encoding
        ctype, encoding = mimetypes.guess_type(filepath)
        if ctype is None or encoding is not None:
            # No guess could be made, or the file is encoded (compressed), so
            # use a generic bag-of-bits type.
            ctype = "application/octet-stream"
        maintype, subtype = ctype.split("/", 1)

        email.add_attachment(data, maintype=maintype, subtype=subtype)
        print(f"Successfully attached: {filepath}")
        return email


    email = EmailMessage()
    email["Sender"] = sender
    email["Recipients"] = ", ".join(recipients)
    email["Subject"] = subject
    email.set_content(body)
    if filepath != None:
        email = add_attachment(email, filepath)

    # Email failsafe.
    confirm_send = input(f"Are you sure you want to send to recipients? (Y/N) \n\n {recipients}\n")

    if confirm_send.lower() == "y":
        context = ssl.create_default_context()
        print("Sending email...")
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp_server:
            smtp_server.login(sender, password)
            smtp_server.sendmail(sender, recipients, email.as_string())
        print("Message sent!")
    else:
        print("Message NOT sent!")


    return email

password = credentials["gmail"]["password"]
email = send_mail(sender, recipients, subject, body, password)


Successfully attached: SF_Ryu.png
Sending email...
Message sent!


# Scraping / Using Selenium

In [191]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

opts = Options()
opts.add_argument("--headless")
browser = Chrome(options=opts)

# try browsing to your gmail account.
url = "https://mail.google.com/mail/u/0/#inbox"
browser.get(url)


gmailId = "jaime.meriz13@gmail.com"
login_box = browser.find_element("xpath", '//*[@id ="identifierId"]')
login_box.send_keys(gmailId)

next_button = browser.find_element("xpath", '//*[@id ="identifierNext"]')
next_button.click()

password = "ABRAZOSrotos2009*" # config["credentials"]["gmail"]["password"]
password_box = browser.find_element("xpath", '//*[@id ="password"]')
# password_box.send_keys(password)

# nextButton = browser.find_element("xpath", '//*[@id ="passwordNext"]')
# nextButton.click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id ="password"]"}
  (Session info: headless chrome=116.0.5845.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001051a665c chromedriver + 4318812
1   chromedriver                        0x000000010519ed00 chromedriver + 4287744
2   chromedriver                        0x0000000104dd07ec chromedriver + 296940
3   chromedriver                        0x0000000104e0e048 chromedriver + 548936
4   chromedriver                        0x0000000104e46d28 chromedriver + 781608
5   chromedriver                        0x0000000104e02178 chromedriver + 500088
6   chromedriver                        0x0000000104e02fc0 chromedriver + 503744
7   chromedriver                        0x0000000105166c40 chromedriver + 4058176
8   chromedri

In [190]:
browser.__dict__

{'vendor_prefix': 'goog',
 'service': <selenium.webdriver.chrome.service.Service at 0x1512a2710>,
 'command_executor': <selenium.webdriver.chromium.remote_connection.ChromiumRemoteConnection at 0x1512957d0>,
 '_is_remote': False,
 'session_id': 'a67ab9a00af123060cc5a5e79d1aadab',
 'caps': {'acceptInsecureCerts': False,
  'browserName': 'chrome',
  'browserVersion': '116.0.5845.140',
  'chrome': {'chromedriverVersion': '116.0.5845.96 (1a391816688002153ef791ffe60d9e899a71a037-refs/branch-heads/5845@{#1382})',
   'userDataDir': '/var/folders/v3/j0_twvc12kb14hzn7ps4qy340000gn/T/.org.chromium.Chromium.Q8knKZ'},
  'fedcm:accounts': True,
  'goog:chromeOptions': {'debuggerAddress': 'localhost:57429'},
  'networkConnectionEnabled': False,
  'pageLoadStrategy': 'normal',
  'platformName': 'mac os x',
  'proxy': {},
  'setWindowRect': True,
  'strictFileInteractability': False,
  'timeouts': {'implicit': 0, 'pageLoad': 300000, 'script': 30000},
  'unhandledPromptBehavior': 'dismiss and notify',


In [178]:
config["credentials"]["gmail"]["password"]

'yimpasqbjgztrfqb'

NameError: name 'WebDriver' is not defined